In [35]:
import os
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain, SequentialChain
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.environ.get("SERPAPI_API_KEY")

### For Single Input and Single Output Chain

In [36]:
llm = OpenAI(
    temperature = 0.6
)

In [37]:
restaurant_prompt = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for my restaurant."
)

In [38]:
restaurant_prompt.format(cuisine = "Indian")

'I want to open a restaurant for Indian food. Suggest a fancy name for my restaurant.'

In [39]:
name_chain = LLMChain(
    llm = llm,
    prompt = restaurant_prompt
)
name_chain.run("Indo-Chinese")

'\n\n"Dragon\'s Delight"'

In [40]:
menu_prompt = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it as a comma separated list."
)

In [41]:
menu_chain = LLMChain(
    llm = llm,
    prompt = menu_prompt
)

In [42]:
chain = SimpleSequentialChain(
    chains = [name_chain, menu_chain]
)
response = chain.run("Brazilian")
print(response)



1. Feijoada (Brazilian black bean stew)
2. Picanha (grilled top sirloin steak)
3. Moqueca (Brazilian seafood stew)
4. Coxinha (fried chicken croquettes)
5. Brigadeiro (chocolate truffles)
6. Acarajé (black-eyed pea fritters)
7. Churrasco (Brazilian-style BBQ)
8. Açaí bowl (blended açaí topped with fruit and granola)
9. Farofa (toasted cassava flour)
10. Caipirinha (Brazilian cocktail with cachaça, lime, and sugar)
11. Cuscuz (steamed cornmeal with toppings like cheese, meat, or vegetables)
12. Pão de queijo (cheese bread rolls)
13. Bobó de camarão (creamy shrimp and cassava stew)
14. Quindim (coconut custard)
15. Pastel (fried pastry filled with meat, cheese, or vegetables)


### For Multiple inputs and Multiple Output Chains

In [43]:
llm = OpenAI(
    temperature = 0.6
)

In [44]:
restaurant_prompt = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for my restaurant."
)

In [45]:
name_chain = LLMChain(
    llm = llm,
    prompt = restaurant_prompt,
    output_key = "restaurant_name"
)

In [46]:
menu_prompt = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it as a comma separated list."
)

In [47]:
menu_chain = LLMChain(
    llm = llm,
    prompt = menu_prompt,
    output_key = "menu_items"
)

In [48]:
chain = SequentialChain(
    chains = [name_chain, menu_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)

response = chain({'cuisine': 'Brazilian'})
print(response)

{'cuisine': 'Brazilian', 'restaurant_name': '\n\n"Sabores do Brasil" (Flavors of Brazil)', 'menu_items': '\n\n1. Feijoada (Brazilian black bean stew)\n2. Coxinha (fried chicken croquettes)\n3. Picanha (grilled beef sirloin)\n4. Moqueca (seafood stew)\n5. Acarajé (black-eyed pea fritters)\n6. Churrasco (Brazilian-style barbecue)\n7. Brigadeiro (chocolate truffles)\n8. Caipirinha (Brazilian cocktail)\n9. Pastel de queijo (cheese pastries)\n10. Vatapá (spicy shrimp and coconut stew)'}


### LLM Agents

In [49]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import OpenAI

In [50]:
tools = load_tools(
    ["wikipedia", "llm-math"],
    llm = llm
)

In [51]:
agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

In [52]:
agent.run("When was Apple founded and How old is Apple Inc. in 2024?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Apple Inc.
Action: Wikipedia
Action Input: Apple Inc.
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I should use Calculator to calculate the age of Apple Inc. in 2024.
Action: Calculator
Action Input: 2024 - 1976
Observation: Answer: 48
Thought: I now know the final answer
Final Answer: In 2024, Apple Inc. will be 48 years old.

> Finished chain.


'In 2024, Apple Inc. will be 48 years old.'

In [54]:
tools = load_tools(
    ["serpapi", "llm-math"],
    llm = llm
)

In [59]:
agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

In [68]:
agent.run("What was the revenue of apple + microsoft in FY22?")



> Entering new AgentExecutor chain...
 To answer this question, I should look for a source that has information on the revenue of both companies in FY22.
Action: Search
Action Input: "apple microsoft revenue FY22"
Observation: Comparatively, in the fiscal year of 2022, hardware-focused Apple's 394.33 billion U.S. dollar revenue was almost double the amount of Microsoft's 198.27 billion U.S. dollars.
Thought: This gives me an idea of the general numbers, but I should verify the specific amounts.
Action: Calculator
Action Input: 394.33 + 198.27
Observation: Answer: 592.6
Thought: I now know the final answer
Final Answer: The combined revenue of Apple and Microsoft in FY22 was 592.6 billion U.S. dollars.

> Finished chain.


'The combined revenue of Apple and Microsoft in FY22 was 592.6 billion U.S. dollars.'

### Memory